Checking the goal using the distance and numpy

In [2]:
##MODEL TO DETECT GOALPOSTS
from inference import get_model

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

GOAL_DETECTION_MODEL_ID = "football-goalpost/3"
GOAL_DETECTION_MODEL = get_model(model_id=GOAL_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

[04/09/25 14:19:40] WARNING  Your inference package version 0.43.0 is out of date! Please upgrade to ]8;id=18926;file://c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\inference\core\__init__.py\__init__.py]8;;\:]8;id=580363;file://c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\inference\core\__init__.py#41\41]8;;\
                             version 0.46.1 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

ModelDependencyMissing: Your `inference` configuration does not support PaliGemma model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Florence2 model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[clip]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does

In [3]:
##MODEL TO DETECT PLAYERS AND BALL AND REFREES
from inference import get_model

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/12"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

In [4]:
##MODEL TO CLASSIFY GOAL OR NO GOAL MOMMENT 
from inference import get_model

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

CLASSIFICATION_MODEL_ID = "goal-ckeck-classification/4"
CLASSIFICATION__MODEL = get_model(model_id=CLASSIFICATION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

In [5]:
import cv2
import numpy as np  
import supervision as sv

In [6]:
image_path = "C:/Users/user/Desktop/FootballAI/videos/ronaldo1.png"  # Path to the image


In [6]:
#DETECT THE GOALPOST SPERATELY

goalpost_center = None  # Variable to store the center of the goalpost

image = cv2.imread(image_path)  # Load the image using OpenCV

result = GOAL_DETECTION_MODEL.infer(image, confidence=0.3)[0]

detections = sv.Detections.from_inference(result)
print(detections.xyxy)

for detection, class_id in zip(detections.xyxy, detections.class_id):
        x_min, y_min, x_max, y_max = detection
        goalpost_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Calculate the center of the goalpost
        print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates
        # Draw a red dotted circle at the center
        center_coordinates = (int(goalpost_center[0]), int(goalpost_center[1]))
        radius = 5
        color = (0, 0, 255)  # Red color in BGR
        thickness = 2
        line_type = cv2.LINE_AA  # Anti-aliased line for better quality
        cv2.circle(image, center_coordinates, radius, color, thickness, lineType=line_type)
        
        break  # Stop after finding the first goalpost

# Display the image with the red dotted circle
cv2.imshow("Image with Center Point", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



[[ 892.  110. 1234.  415.]]
Goalpost center: (1063.0, 262.5)


In [7]:
#DETECT THE BALL SPERATELY

ball_center = None  # Variable to store the center of the ball
image = cv2.imread(image_path)  # Load the image using OpenCV
BALL_ID = 0
result = PLAYER_DETECTION_MODEL.infer(image, confidence=0.2)[0]

detections = sv.Detections.from_inference(result)

for detection, class_id in zip(detections.xyxy, detections.class_id):
    if class_id == BALL_ID:
        x_min, y_min, x_max, y_max = detection
        ball_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Calculate the center of the ball
        print(f"Ball center: {ball_center}")  # Print the center coordinates

# Check if ball_center is not None before using it
if ball_center is not None:
    center_coordinates = (int(ball_center[0]), int(ball_center[1]))
    radius = 5
    thickness = 2
    line_type = cv2.LINE_AA  # Anti-aliased line for better quality
    cv2.circle(image, center_coordinates, radius, (0, 0, 255), thickness, line_type)  # Draw the circle

cv2.imshow("Image with Center Point", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Ball center: (866.5, 272.0)
Ball center: (1352.5, 406.0)


In [56]:
# Draw a red dotted circle at the center of the goalpost
if goalpost_center is not None:
    goalpost_center_coordinates = (int(goalpost_center[0]), int(goalpost_center[1]))
    cv2.circle(image, goalpost_center_coordinates, radius, (0, 0, 255), thickness, lineType=line_type)

# Draw a red dotted circle at the center of the ball
if ball_center is not None:
    ball_center_coordinates = (int(ball_center[0]), int(ball_center[1]))
    cv2.circle(image, ball_center_coordinates, radius, (0, 0, 255), thickness, lineType=line_type)

# Display the image with both circles
cv2.imshow("Image with Goalpost and Ball Centers", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates of the goalpost
print(f"Ball center: {ball_center}")  # Print the center coordinates of the ball

Goalpost center: (1629.5, 208.5)
Ball center: (1191.0, 248.0)


In [116]:
# Calculate the Euclidean distance between goalpost_center and ball_center
distance = np.sqrt((goalpost_center[0] - ball_center[0])**2 + (goalpost_center[1] - ball_center[1])**2)

# Define a threshold for "close enough" (you can adjust this value as needed)
threshold =50  # Example threshold

# Determine if it's a goal
if distance <= threshold:
    result = "Goal"
else:
    result = "Not Goal"

print(f"Distance: {distance}")
print(f"Result: {result}")

Distance: 322.90130070967507
Result: Not Goal


In [7]:

    
    # If the ball is close enough to the goal, verify with classification model
def classify_goal():
        classification_result = CLASSIFICATION__MODEL.infer(image)  # Pass the image to the classifier
        goal_prediction = classification_result[0]  # Access the first prediction from the list
        goal_label = goal_prediction.predicted_classes[0]  # Get the predicted class
        goal_probability = goal_prediction.predictions[goal_label].confidence  # Get the confidence score
        return goal_label,goal_probability

goal_label, goal_probability = classify_goal()  # Use the correct attribute for the confidence score
    
    print(f"Classification Model Prediction: {goal_label} (Confidence: {goal_probability * 100:.2f}%)")
    
    if goal_label == "goal":
        print("Goal detected!")
    elif goal_label == "nogoal":
        print("No Goal detected.")
    else:
        print("Ball or goalpost not detected.")

NameError: name 'image' is not defined

In [ ]:
# Path to the video file
video_path = "C:/Users/user/Desktop/FootballAI/videos/english.mp4"

# Path to save the resulting video
output_video_path = "C:/Users/user/Desktop/FootballAI/videos/resulting_video5.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize the goal counter
goal_counter = 0

# Loop over frames in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if no more frames are available

    # Detect goalpost and ball in the current frame
   
      
    # Use the classification model to double-check
    classification_result = CLASSIFICATION__MODEL.infer(frame)  # Pass the image to the classifier
    goal_prediction = classification_result[0]  # Access the first prediction from the list
    goal_label = goal_prediction.predicted_classes[0]  # Get the predicted class
    goal_probability = goal_prediction.predictions[goal_label].confidence  # Get the confidence score

    if goal_label == "goal" and goal_probability > 0.8 and goal_counter == 0: #Confidence threshold for classification
           goal_counter += 1 # Increment the goal counter

    # Display the frame with the goal counter
    cv2.putText(frame, f"Goals Scored: {goal_counter}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    # Write the frame to the output video
    out.write(frame)

    # Optionally display the frame (can be commented out if not needed)
    cv2.imshow("Video", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects, and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Resulting video saved at: {output_video_path}")

Resulting video saved at: C:/Users/user/Desktop/FootballAI/videos/resulting_video5.mp4


In [ ]:
 goalpost_center = None
    ball_center = None

    # Infer goalpost
    goalpost_result = GOAL_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    goalpost_detections = sv.Detections.from_inference(goalpost_result)
    for detection in goalpost_detections.xyxy:
        x_min, y_min, x_max, y_max = detection
        goalpost_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)
        break  # Stop after finding the first goalpost

    # Infer ball
    ball_result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
    ball_detections = sv.Detections.from_inference(ball_result)
    for detection, class_id in zip(ball_detections.xyxy, ball_detections.class_id):
        if class_id == BALL_ID:
            x_min, y_min, x_max, y_max = detection
            ball_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)
            break  # Stop after finding the ball

    # Check if both goalpost and ball are detected
    if goalpost_center and ball_center:

In [ ]:
To improve the accuracy of detection in your current setup, you can consider the following strategies:

1. **Fine-tune the Model**:
    - Use a dataset that closely matches your use case and fine-tune the pre-trained model on it. This ensures the model learns features specific to your application.

2. **Increase the Confidence Threshold**:
    - Adjust the confidence threshold in the `infer` method to reduce false positives. For example:
      ```python
      result = GOAL_DETECTION_MODEL.infer(image, confidence=0.5)[0]
      ```

3. **Use a Higher-Resolution Image**:
    - Ensure the input image has a higher resolution to provide more details for the model to detect objects accurately.

4. **Data Augmentation**:
    - Augment your training dataset with techniques like rotation, flipping, cropping, and brightness adjustments to make the model more robust.

5. **Non-Maximum Suppression (NMS) Tuning**:
    - Adjust the NMS threshold to better handle overlapping detections. For example:
      ```python
      detections = sv.Detections.from_inference(result, nms_threshold=0.4)
      ```

6. **Use a More Advanced Model**:
    - If possible, switch to a more advanced version of the YOLO model or another state-of-the-art object detection model like Faster R-CNN or EfficientDet.

7. **Post-Processing**:
    - Apply additional filtering logic based on object size, aspect ratio, or spatial relationships to refine detections.

8. **Improve Dataset Quality**:
    - Ensure your dataset is well-labeled and diverse, covering various scenarios, lighting conditions, and angles.

9. **Ensemble Models**:
    - Combine predictions from multiple models to improve accuracy. For example, use both YOLO and Faster R-CNN and take a consensus of their predictions.

10. **Use Region of Interest (ROI)**:
     - Focus detection on specific areas of the image where the objects of interest are likely to appear, reducing noise from irrelevant regions.

11. **Hyperparameter Tuning**:
     - Experiment with hyperparameters like learning rate, batch size, and number of epochs during training to optimize the model's performance.

12. **Regularly Update the Model**:
     - Retrain the model periodically with new data to adapt to changing conditions or scenarios.

By implementing these strategies, you can significantly enhance the accuracy and reliability of your object detection system.

In [ ]:
# Detect players and ball
result = PLAYER_DETECTION_MODEL.infer(image, confidence=0.29)[0]
detections = sv.Detections.from_inference(result)

ball_center = None
players = []

# Extract ball and player detections
for detection, class_id, class_name in zip(detections.xyxy, detections.class_id, detections.data['class_name']):
    x_min, y_min, x_max, y_max = detection
    center = ((x_min + x_max) / 2, (y_min + y_max) / 2)
    
    if class_id == BALL_ID:  # Ball detection
        ball_center = center
    elif class_name == "player":  # Player detection
        players.append({'center': center, 'team': None})  # Add team info if available

# Assign teams to players (e.g., based on jersey color)
for player in players:
    x, y = map(int, player['center'])
    color = image[y, x]  # Get the pixel color at the player's center
    if color[0] > color[2]:  # Example logic: blue team vs red team
        player['team'] = 'Team A'
    else:
        player['team'] = 'Team B'

# Find the player closest to the ball
if ball_center:
    closest_player = min(players, key=lambda p: np.sqrt((p['center'][0] - ball_center[0])**2 + (p['center'][1] - ball_center[1])**2))
    scoring_team = closest_player['team']
    print(f"The scoring team is: {scoring_team}")
else:
    print("Ball not detected.")
